In [16]:
import pandas as pd
from dont_patronize_me import DontPatronizeMe
from ast import literal_eval
from torch.utils.data import DataLoader, Dataset
import os.path as osp
from typing import Optional
from transformer import RoBERTa

import pandas as pd
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
from transformers.models.auto.tokenization_auto import AutoTokenizer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = 'cuda'

In [8]:
class DPMDataset_submission(Dataset):
    def __init__(self, model: str, data) -> None:
        super().__init__()

        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.max_len = 512
        self.data = data

    def __getitem__(self, index):
        text = self.data[index]
        text = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True,
        )
        return {
            "ids": torch.tensor(text["input_ids"], dtype=torch.long),
            "mask": torch.tensor(text["attention_mask"], dtype=torch.long),
        }

    def __len__(self):
        return len(self.data)

In [5]:
dpm = DontPatronizeMe('../dataset', test_path="../dataset/task4_test.tsv")
dpm.load_test()

In [9]:
dataset_ = DPMDataset_submission(model = "mrm8488/distilroberta-finetuned-tweets-hate-speech",data=dpm.test_set)

In [10]:
data_loader = DataLoader(dataset_, batch_size=8)

In [ ]:
best_model_path = " "
module = RoBERTa.load_from_checkpoint(best_model_path)

module.eval()
module.freeze()
preds = []
file_path = "../submissions/task2.txt"
for batch in data_loader:
    img, img_path = batch
    out = module(img)
    predictions = torch.argmax(out, 1).numpy()
    for i, pred in enumerate(predictions):
        preds.append(pred)
result_df = pd.DataFrame(preds, columns=None)
result_df.to_csv(file_path, index=False)
print("submission created")